In [1]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 44.4 MB/s eta 0:00:00


In [2]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

In [3]:
%pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 2.8 MB/s eta 0:00:00


In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.document_loaders import PyPDFLoader

from langchain_community.vectorstores import Chroma
from langchain_text_splitters import CharacterTextSplitter

# load the document and split it into chunks
# pdf_loader = PyPDFLoader('Stress.pdf')
# documents = pdf_loader.load()

infosheets = [
    "Stress (1).pdf",
    "Dementia.pdf",
    "Music Therapy.pdf",
    "Bullying in Children and Youth Guide for Caregivers.pdf",
    "Attention Deficit  Hyperactivity Disorder ADHD in Adults.pdf"
]

documents = []

for infosheet in infosheets:
  pdf_loader = PyPDFLoader(infosheet)
  current_info_sheet = pdf_loader.load()
  documents += (current_info_sheet)

In [5]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 9.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 5

In [6]:

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

# query it
query = "What is causing my stress?"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Page 2 / 3
skyrocketing.
Work – can be an ever-present source of stress. Work stress is caused by things such as job dissatisfaction,
an exhausting workload, insuﬃcient pay, oﬃce politics, and conﬂicts with your boss or co-workers. 
Life situations – can cause stress. For example, poverty, ﬁnancial pressures, discrimination or harassment,
unemployment, isolation, and/or a lack of social support all take a toll on your daily quality of life.
Major life changes — such as the birth of a baby, a divorce, a career change or move can also place a lot of
stress on you—even if the event itself is positive.
In small amounts, stress is actually good for us. Stress can motivate us and push us to reach our potential. It’s
stress that helps you get through that presentation to your clients or forces us to do homework when we’d rather
take a nap. Stress is very individual. What you ﬁnd stressful may not be stressful for someone else.
 
It’s important to know that stress in itself is not a mental ill

In [7]:
# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cuda:0'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
text = "This is a test document."
query_result = embeddings.embed_query(text)
query_result[:3]


[-0.0383385568857193, 0.12346474081277847, -0.028642933815717697]

In [9]:
db = FAISS.from_documents(docs, embeddings)


In [10]:
question = "What is stress?"
searchDocs = db.similarity_search(question)
print(searchDocs[0].page_content)

Page 2 / 3
skyrocketing.
Work – can be an ever-present source of stress. Work stress is caused by things such as job dissatisfaction,
an exhausting workload, insuﬃcient pay, oﬃce politics, and conﬂicts with your boss or co-workers. 
Life situations – can cause stress. For example, poverty, ﬁnancial pressures, discrimination or harassment,
unemployment, isolation, and/or a lack of social support all take a toll on your daily quality of life.
Major life changes — such as the birth of a baby, a divorce, a career change or move can also place a lot of
stress on you—even if the event itself is positive.
In small amounts, stress is actually good for us. Stress can motivate us and push us to reach our potential. It’s
stress that helps you get through that presentation to your clients or forces us to do homework when we’d rather
take a nap. Stress is very individual. What you ﬁnd stressful may not be stressful for someone else.
 
It’s important to know that stress in itself is not a mental ill

In [11]:
# Create a tokenizer object by loading the pretrained "Intel/dynamic_tinybert" tokenizer.
tokenizer = AutoTokenizer.from_pretrained("Intel/dynamic_tinybert")

# Create a question-answering model object by loading the pretrained "Intel/dynamic_tinybert" model.
model = AutoModelForQuestionAnswering.from_pretrained("Intel/dynamic_tinybert")

tokenizer_config.json:   0%|          | 0.00/351 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [12]:
model_name = "Intel/dynamic_tinybert"

# Load the tokenizer associated with the specified model
tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, truncation=True, max_length=512)

# Define a question-answering pipeline using the model and tokenizer
question_answerer = pipeline(
    "question-answering",
    model=model_name,
    tokenizer=tokenizer,
    return_tensors='pt'
)

# Create an instance of the HuggingFacePipeline, which wraps the question-answering pipeline
# with additional model-specific arguments (temperature and max_length)
llm = HuggingFacePipeline(
    pipeline=question_answerer,
    model_kwargs={"temperature": 0.7, "max_length": 512},
)

In [13]:
retriever = db.as_retriever()


In [14]:
docs = retriever.get_relevant_documents("What is stress?")
print(docs[0].page_content)


Page 2 / 3
skyrocketing.
Work – can be an ever-present source of stress. Work stress is caused by things such as job dissatisfaction,
an exhausting workload, insuﬃcient pay, oﬃce politics, and conﬂicts with your boss or co-workers. 
Life situations – can cause stress. For example, poverty, ﬁnancial pressures, discrimination or harassment,
unemployment, isolation, and/or a lack of social support all take a toll on your daily quality of life.
Major life changes — such as the birth of a baby, a divorce, a career change or move can also place a lot of
stress on you—even if the event itself is positive.
In small amounts, stress is actually good for us. Stress can motivate us and push us to reach our potential. It’s
stress that helps you get through that presentation to your clients or forces us to do homework when we’d rather
take a nap. Stress is very individual. What you ﬁnd stressful may not be stressful for someone else.
 
It’s important to know that stress in itself is not a mental ill

In [20]:
model_name = "Intel/dynamic_tinybert"

# Load the tokenizer associated with the specified model
tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, truncation=True, max_length=512)

# Define a question-answering pipeline using the model and tokenizer
question_answerer = pipeline(
    "question-answering",
    model=model_name,
    tokenizer=tokenizer,
    return_tensors='pt'
)

# Create an instance of the HuggingFacePipeline, which wraps the question-answering pipeline
# with additional model-specific arguments (temperature and max_length)
llm = HuggingFacePipeline(
    pipeline=question_answerer,
    model_kwargs={"temperature": 0.7, "max_length": 512},
)
# Create a retriever object from the 'db' with a search configuration where it retrieves up to 4 relevant splits/documents.
retriever = db.as_retriever(search_kwargs={"k": 4})

# Create a question-answering instance (qa) using the RetrievalQA class.
# It's configured with a language model (llm), a chain type "refine," the retriever we created, and an option to not return source documents.
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever, return_source_documents=False)
question = "What is stress?"
context = "Stress is the body's reaction to any change that requires an adjustment or response. The body reacts to these changes with physical, mental, and emotional responses. Stress is a normal part of life. You can experience stress from your environment, your body, and your thoughts."

result = qa.invoke({"query": question, "context": context})
print(result["result"])

ValueError: Context information is below. 
------------
Page 2 / 3
skyrocketing.
Work – can be an ever-present source of stress. Work stress is caused by things such as job dissatisfaction,
an exhausting workload, insuﬃcient pay, oﬃce politics, and conﬂicts with your boss or co-workers. 
Life situations – can cause stress. For example, poverty, ﬁnancial pressures, discrimination or harassment,
unemployment, isolation, and/or a lack of social support all take a toll on your daily quality of life.
Major life changes — such as the birth of a baby, a divorce, a career change or move can also place a lot of
stress on you—even if the event itself is positive.
In small amounts, stress is actually good for us. Stress can motivate us and push us to reach our potential. It’s
stress that helps you get through that presentation to your clients or forces us to do homework when we’d rather
take a nap. Stress is very individual. What you ﬁnd stressful may not be stressful for someone else.
 
It’s imp